# HPCSeries Core v0.7 - Getting Started

Welcome to HPCSeries Core! This notebook provides a quick introduction to using the high-performance statistical computing library.

## What is HPCSeries Core?

HPCSeries Core is a high-performance statistical computing library that provides:
- **SIMD-accelerated operations** for maximum performance
- **Automatic parallelization** with OpenMP
- **C/Fortran backend** for computationally intensive operations
- **Python API** for ease of use
- **Auto-tuning calibration** to optimize for your hardware

## Installation

```bash
# Install in development mode
pip install -e .
```

## Quick Start

In [ ]:
import hpcs
import numpy as np
import time

# Display version and CPU info
print(f"HPCSeries Core version: {hpcs.__version__}")
print("\nRunning CPU detection...")
!hpcs cpuinfo

## 1. Basic Statistical Operations

HPCSeries provides fast implementations of common statistical functions:

In [ ]:
# Create sample data
data = np.random.randn(1_000_000)

print("Basic Statistics:")
print(f"Sum:      {hpcs.sum(data):.6f}")
print(f"Mean:     {hpcs.mean(data):.6f}")
print(f"Std Dev:  {hpcs.std(data):.6f}")
print(f"Variance: {hpcs.var(data):.6f}")
print(f"Min:      {hpcs.min(data):.6f}")
print(f"Max:      {hpcs.max(data):.6f}")

## 2. Robust Statistics

Robust statistics are less sensitive to outliers:

In [ ]:
# Add some outliers
data_with_outliers = np.concatenate([data, [100, -100, 200, -200]])

print("Comparison: Regular vs Robust Statistics")
print(f"\nMean:          {hpcs.mean(data_with_outliers):.6f}")
print(f"Median:        {hpcs.median(data_with_outliers):.6f}")
print(f"\nStd Dev:       {hpcs.std(data_with_outliers):.6f}")
print(f"MAD (robust):  {hpcs.mad(data_with_outliers):.6f}")

## 3. Rolling Window Operations

Compute statistics over sliding windows (v0.7 features C-accelerated rolling operations):

In [ ]:
# Create time series data
time_series = np.cumsum(np.random.randn(10000))
window = 50

# Rolling statistics
rolling_mean = hpcs.rolling_mean(time_series, window)
rolling_std = hpcs.rolling_std(time_series, window)
rolling_median = hpcs.rolling_median(time_series, window)

print(f"Computed rolling statistics with window={window}")
print(f"Output shape: {rolling_mean.shape}")
print(f"First {window-1} values are NaN (insufficient data)")
print(f"Valid values start at index {window-1}")

## 4. Z-Score Normalization (NEW in v0.7)

Fast C-accelerated z-score computation:

In [ ]:
# Rolling z-score (C-optimized in v0.7)
rolling_zscore = hpcs.rolling_zscore(time_series, window)

# Robust z-score using MAD
rolling_robust_zscore = hpcs.rolling_robust_zscore(time_series, window)

print(f"Rolling z-score computed (window={window})")
print(f"Mean of z-scores: {np.nanmean(rolling_zscore):.6f} (should be ~0)")
print(f"Std of z-scores:  {np.nanstd(rolling_zscore):.6f} (should be ~1)")

## 5. Performance Comparison

Let's compare HPCSeries with NumPy:

In [ ]:
# Large dataset for meaningful benchmarks
large_data = np.random.randn(10_000_000)

# Benchmark sum
start = time.perf_counter()
result_hpcs = hpcs.sum(large_data)
time_hpcs = time.perf_counter() - start

start = time.perf_counter()
result_numpy = np.sum(large_data)
time_numpy = time.perf_counter() - start

print(f"Sum of {len(large_data):,} elements:")
print(f"  HPCSeries: {time_hpcs*1000:.3f} ms")
print(f"  NumPy:     {time_numpy*1000:.3f} ms")
print(f"  Speedup:   {time_numpy/time_hpcs:.2f}x")
print(f"  Results match: {np.allclose(result_hpcs, result_numpy)}")

## 6. Auto-Tuning Calibration (NEW in v0.7)

HPCSeries can automatically calibrate to your hardware for optimal performance:

In [ ]:
# Run quick calibration (5-10 seconds)
print("Running quick calibration...")
hpcs.calibrate(quick=True)
print("\nCalibration complete!")

# Save configuration
import os
config_path = os.path.expanduser("~/.hpcs/config.json")
hpcs.save_calibration_config(config_path)
print(f"Configuration saved to: {config_path}")

## 7. CLI Commands

HPCSeries also provides a command-line interface:

In [ ]:
# Show version
!hpcs version

# Show CPU information
!hpcs cpuinfo

# Run quick calibration
!hpcs calibrate --quick

# Run benchmarks
!hpcs bench --size 1000000 --iterations 5

## Next Steps

Explore more advanced features:

1. **01_rolling_mean_vs_median.ipynb** - Rolling window operations
2. **02_robust_anomaly_climate.ipynb** - Anomaly detection with robust statistics
3. **03_batched_iot_rolling.ipynb** - Batched processing for IoT data
4. **04_axis_reductions_column_stats.ipynb** - 2D array operations
5. **05_masked_missing_data.ipynb** - Handling missing data
6. **06_performance_calibration.ipynb** - Performance tuning and optimization
7. **07_c_optimized_operations.ipynb** - C-accelerated operations (v0.7)

## Documentation

- **GitHub**: [HPCSeries Core Repository](https://github.com/yourusername/HPCSeriesCore)
- **API Reference**: See `docs/` directory
- **Specifications**: See root directory for detailed specs

## Support

For issues, questions, or contributions, please visit the GitHub repository.